In [6]:
import tensorflow as tf
import nltk

In [2]:
tf.__version__

'2.3.0'

In [40]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
import pandas as pd
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy

In [4]:
voc_size = 5000

In [7]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [11]:
df = pd.read_csv('train.csv')

In [12]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [22]:
import re
from nltk.corpus import stopwords

In [18]:
a = df['title'][0]
type(a)

str

In [24]:
# Text Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0,len(df)):
  ps = PorterStemmer()
  review = re.sub('[^a-zA-Z]',' ', str(df['title'][i]))
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [29]:
corpus[0]

'hous dem aid even see comey letter jason chaffetz tweet'

In [28]:
one_hot_repre = [one_hot(word,voc_size) for word in corpus]
one_hot_repre

[[4335, 648, 2567, 453, 22, 4829, 1785, 2655, 2961, 1338],
 [1145, 4021, 2145, 3918, 556, 3835, 501],
 [2616, 4278, 1354, 535],
 [761, 1909, 3613, 1731, 2163, 2388],
 [1704, 556, 498, 4949, 2314, 1230, 556, 4066, 3112, 457],
 [2279,
  1837,
  4462,
  4810,
  1088,
  2058,
  219,
  1524,
  2650,
  2658,
  2201,
  386,
  1514,
  2973,
  501],
 [4057, 4057, 3633, 760, 3091, 1771, 3435, 4993, 958, 2356, 3067, 4326],
 [1564, 1389, 3422, 1735, 126, 21, 2708, 1862, 784, 4804, 3200],
 [1398, 4337, 1600, 1293, 2058, 3127, 2974, 2127, 18, 3313, 784, 4804, 3200],
 [2310, 2078, 91, 4788, 3963, 1850, 2058, 4915, 784, 4804, 3200],
 [2186, 826, 325, 1873, 1961, 4309, 4602, 1629, 2058, 3502],
 [2106, 4587, 2281, 2828, 1523, 1034, 4359, 3041],
 [4964, 142, 2007, 1322, 4613, 2460, 879, 1492, 3607, 3110, 2518],
 [1731, 2713, 22, 4309, 2058, 3963],
 [4609, 2836, 3157, 2982, 3357, 782, 2293, 4242, 4104],
 [3855, 1171, 2974, 1365, 1599, 2168, 4153, 784, 4804, 3200],
 [4548, 4318, 2250, 3487, 2133, 784, 4804

In [31]:
sent_length = 20 
embed_docs = pad_sequences(one_hot_repre,sent_length,padding='pre')
print(embed_docs)

[[   0    0    0 ... 2655 2961 1338]
 [   0    0    0 ...  556 3835  501]
 [   0    0    0 ... 4278 1354  535]
 ...
 [   0    0    0 ...  784 4804 3200]
 [   0    0    0 ...  468 1118 4288]
 [   0    0    0 ...  302 2209 4146]]


In [46]:
dim = 40
model = Sequential()
model.add(Embedding(voc_size,dim,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss=binary_crossentropy,optimizer='adam',metrics=['accuracy'])

In [47]:
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [35]:
import numpy as np
X = np.array(embed_docs)
y = np.array(df['label'])

In [36]:
X.shape

(20800, 20)

In [37]:
y.shape

(20800,)

In [38]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=45)


In [48]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
228/228 [==============================] - 7s 32ms/step - loss: 0.3057 - accuracy: 0.8575 - val_loss: 0.2026 - val_accuracy: 0.9183
Epoch 2/10
228/228 [==============================] - 7s 30ms/step - loss: 0.1349 - accuracy: 0.9456 - val_loss: 0.1752 - val_accuracy: 0.9293
Epoch 3/10
228/228 [==============================] - 7s 31ms/step - loss: 0.0980 - accuracy: 0.9641 - val_loss: 0.1792 - val_accuracy: 0.9293
Epoch 4/10
228/228 [==============================] - 7s 31ms/step - loss: 0.0690 - accuracy: 0.9753 - val_loss: 0.2167 - val_accuracy: 0.9263
Epoch 5/10
228/228 [==============================] - 7s 30ms/step - loss: 0.0458 - accuracy: 0.9843 - val_loss: 0.3042 - val_accuracy: 0.9139
Epoch 6/10
228/228 [==============================] - 7s 31ms/step - loss: 0.0288 - accuracy: 0.9907 - val_loss: 0.3108 - val_accuracy: 0.9207
Epoch 7/10
228/228 [==============================] - 7s 31ms/step - loss: 0.0185 - accuracy: 0.9945 - val_loss: 0.3981 - val_accuracy: 0.9197

In [49]:
from tensorflow.keras.layers import Dropout

In [51]:
model1 = Sequential()
model1.add(Embedding(voc_size,dim,input_length=sent_length))
model1.add(Dropout(0.3))
model1.add(LSTM(100))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss=binary_crossentropy,optimizer='adam',metrics=['accuracy'])

In [52]:
model1.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
dropout_2 (Dropout)          (None, 20, 40)            0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [53]:
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
228/228 [==============================] - 8s 33ms/step - loss: 0.3211 - accuracy: 0.8468 - val_loss: 0.1719 - val_accuracy: 0.9280
Epoch 2/10
228/228 [==============================] - 7s 31ms/step - loss: 0.1464 - accuracy: 0.9435 - val_loss: 0.1811 - val_accuracy: 0.9316
Epoch 3/10
228/228 [==============================] - 7s 31ms/step - loss: 0.1113 - accuracy: 0.9587 - val_loss: 0.1789 - val_accuracy: 0.9290
Epoch 4/10
228/228 [==============================] - 7s 32ms/step - loss: 0.0838 - accuracy: 0.9707 - val_loss: 0.2041 - val_accuracy: 0.9309
Epoch 5/10
228/228 [==============================] - 7s 31ms/step - loss: 0.0668 - accuracy: 0.9769 - val_loss: 0.2234 - val_accuracy: 0.9274
Epoch 6/10
228/228 [==============================] - 7s 31ms/step - loss: 0.0490 - accuracy: 0.9821 - val_loss: 0.2678 - val_accuracy: 0.9264
Epoch 7/10
228/228 [==============================] - 7s 32ms/step - loss: 0.0368 - accuracy: 0.9875 - val_loss: 0.2698 - val_accuracy: 0.9223

In [54]:
y_pred = model1.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [55]:
y_pred[0:25]

array([[1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1]], dtype=int32)

In [56]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix


In [57]:
accuracy_score(y_test,y_pred)

0.9224358974358975

In [58]:
confusion_matrix(y_test,y_pred)

array([[2860,  285],
       [ 199, 2896]])